In [18]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [20]:
pd.set_option('display.max_columns', None)

games1 = pd.read_csv('../data/raw/games2024.csv')
games2 = pd.read_csv('../data/raw/games2025.csv')
games = pd.concat([games1, games2], ignore_index=True)

In [21]:
games.head()

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore
0,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31
1,2022091100,2022,1,09/11/2022,13:00:00,ATL,NO,26,27
2,2022091101,2022,1,09/11/2022,13:00:00,CAR,CLE,24,26
3,2022091102,2022,1,09/11/2022,13:00:00,CHI,SF,19,10
4,2022091103,2022,1,09/11/2022,13:00:00,CIN,PIT,20,23


In [30]:
df1 = pd.read_csv('../data/raw/plays2024.csv')
data1 = df1[['gameId', 'quarter', 'down', 'yardsToGo', 'gameClock', 'absoluteYardlineNumber', 'possessionTeam', 'defensiveTeam', 'preSnapHomeScore', 'preSnapVisitorScore', 'offenseFormation', 'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability']]#, 'receiverAlignment'
result1 = df1['passResult']

df2 = pd.read_csv('../data/raw/plays2025.csv')
data2 = df2[['gameId', 'quarter', 'down', 'yardsToGo', 'gameClock', 'absoluteYardlineNumber', 'possessionTeam', 'defensiveTeam', 'preSnapHomeScore', 'preSnapVisitorScore', 'offenseFormation', 'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability']]#, 'receiverAlignment'
result2 = df2['passResult']

data = pd.concat([data1, data2], ignore_index=True)
result = pd.concat([result1, result2], ignore_index=True).tolist()

In [31]:
data.head()

,gameId,quarter,down,yardsToGo,gameClock,absoluteYardlineNumber,possessionTeam,defensiveTeam,preSnapHomeScore,preSnapVisitorScore,offenseFormation,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability
0,2022100908,4,1,10,7:52,69,ATL,TB,21,7,SHOTGUN,0.976785,0.023215
1,2022091103,4,1,10,7:38,76,PIT,CIN,14,20,SHOTGUN,0.160485,0.839515
2,2022091111,2,2,5,8:57,40,LV,LAC,10,3,I_FORM,0.756661,0.243339
3,2022100212,3,2,10,13:12,47,DEN,LV,19,16,SINGLEBACK,0.620552,0.379448
4,2022091900,2,1,10,8:33,75,BUF,TEN,7,7,I_FORM,0.836290,0.163710


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

times = []
possessionScore = []
defenceScore = []
pointDiff = []

for i in range(len(data)):
    if data['quarter'][i] in [1, 3]:
        additional = 15 * 60  
    else:
        additional = 0
    time = data['gameClock'][i].split(':')
    times.append(additional + int(time[0]) * 60 + int(time[1]))

    gameid = data['gameId'][i]
    pos = data['possessionTeam'][i]
    defence = data['defensiveTeam'][i]
    row = games[games['gameId'] == gameid]

    if pos == row.iloc[0]['homeTeamAbbr']:
        possessionScore.append(data['preSnapHomeScore'][i])
        defenceScore.append(data['preSnapVisitorScore'][i])
        pointDiff.append(data['preSnapHomeScore'][i] - data['preSnapVisitorScore'][i])
    else:
        possessionScore.append(data['preSnapVisitorScore'][i])
        defenceScore.append(data['preSnapHomeScore'][i])
        pointDiff.append(data['preSnapVisitorScore'][i] - data['preSnapHomeScore'][i])

data['winProbabilityDiff'] = data['preSnapHomeTeamWinProbability'] - data['preSnapVisitorTeamWinProbability']

data['timeLeftHalf'] = times
data['pointDifferential'] = pointDiff

data = data.drop(['gameClock', 'gameId', 'preSnapHomeScore', 'preSnapVisitorScore', 
                  'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability'], axis=1)

label_encoder = LabelEncoder()
data['possessionTeam'] = label_encoder.fit_transform(data['possessionTeam'])
data['defensiveTeam'] = label_encoder.fit_transform(data['defensiveTeam'])
data['offenseFormation'] = label_encoder.fit_transform(data['offenseFormation'])

target = [0 if isinstance(i, str) else 1 for i in result]
target = pd.Series(target, name="target")

print("Processed Data:")
print(data[:10])
print("\nRaw Target Column:")
print(result[:10])
print("\nProcessed Target:")
print(target[:10])

Processed Data:
   quarter  down  yardsToGo  absoluteYardlineNumber  possessionTeam  \
0        4     1         10                      69               1   
1        4     1         10                      76              26   
2        2     2          5                      40              18   
3        3     2         10                      47               9   
4        2     1         10                      75               3   
5        3     3         17                      28               1   
6        4     3          5                      85              30   
7        1     1         10                      85              28   
8        2     1         10                      70              15   
9        3     1         10                      85              27   

   defensiveTeam  offenseFormation  winProbabilityDiff  timeLeftHalf  \
0             29                 4            0.953569           472   
1              6                 4           -0.679031    

In [17]:
import os

# Specify the folder for saving
processed_data_dir = "../data/processed"
os.makedirs(processed_data_dir, exist_ok=True)  # Create the folder if it doesn't exist

# Define file paths
data_file = os.path.join(processed_data_dir, "preprocessed_data.csv")
target_file = os.path.join(processed_data_dir, "target.csv")

# Save processed data
data.to_csv(data_file, index=False)
print(f"Processed data saved to: {data_file}")

# Save target as a separate file
pd.DataFrame(target, columns=["target"]).to_csv(target_file, index=False)
print(f"Target data saved to: {target_file}")

Processed data saved to: ../data/processed/preprocessed_data.csv
Target data saved to: ../data/processed/target.csv
